In [1]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
import numpy as np

In [16]:
columns = ['GAME_ID','HOME_TEAM','AWAY_TEAM','HOME_WIN']

In [17]:
gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable="2024-25",season_type_nullable="Regular Season",league_id_nullable="00")


games_df = gamefinder.get_data_frames()[0]
games_df = games_df.drop_duplicates(subset='GAME_ID')
games_df['IS_HOME'] = games_df['MATCHUP'].str.contains('vs.')
split_matchup = games_df['MATCHUP'].str.replace('@','vs.').str.split('vs.',expand=True)

split_matchup = split_matchup.apply(lambda col: col.str.strip())
games_df['HOME_TEAM'] = split_matchup[0]
games_df['AWAY_TEAM'] = split_matchup[1]
away_mask = ~games_df['IS_HOME']
games_df.loc[away_mask,['HOME_TEAM','AWAY_TEAM']]=games_df.loc[away_mask,['AWAY_TEAM','HOME_TEAM']].values
games_df['HOME_WIN'] = np.where((games_df['IS_HOME'])^(games_df['WL']=='W'),False,True)
games_df = games_df[columns]
games_df.to_csv("games_2024.csv",index=False)

ConnectionError: HTTPSConnectionPool(host='stats.nba.com', port=443): Max retries exceeded with url: /stats/leaguegamefinder?Conference=&DateFrom=&DateTo=&Division=&DraftNumber=&DraftRound=&DraftTeamID=&DraftYear=&EqAST=&EqBLK=&EqDD=&EqDREB=&EqFG3A=&EqFG3M=&EqFG3_PCT=&EqFGA=&EqFGM=&EqFG_PCT=&EqFTA=&EqFTM=&EqFT_PCT=&EqMINUTES=&EqOREB=&EqPF=&EqPTS=&EqREB=&EqSTL=&EqTD=&EqTOV=&GameID=&GtAST=&GtBLK=&GtDD=&GtDREB=&GtFG3A=&GtFG3M=&GtFG3_PCT=&GtFGA=&GtFGM=&GtFG_PCT=&GtFTA=&GtFTM=&GtFT_PCT=&GtMINUTES=&GtOREB=&GtPF=&GtPTS=&GtREB=&GtSTL=&GtTD=&GtTOV=&LeagueID=00&Location=&LtAST=&LtBLK=&LtDD=&LtDREB=&LtFG3A=&LtFG3M=&LtFG3_PCT=&LtFGA=&LtFGM=&LtFG_PCT=&LtFTA=&LtFTM=&LtFT_PCT=&LtMINUTES=&LtOREB=&LtPF=&LtPTS=&LtREB=&LtSTL=&LtTD=&LtTOV=&Outcome=&PORound=&PlayerID=&PlayerOrTeam=T&RookieYear=&Season=2024-25&SeasonSegment=&SeasonType=Regular+Season&StarterBench=&TeamID=&VsConference=&VsDivision=&VsTeamID=&YearsExperience= (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000

In [8]:
team_stats_df = pd.read_csv('master_team_stats_2024.csv')

In [13]:
games_with_home = games_df.merge(
    team_stats_df.add_prefix('HOME_'),
    left_on='HOME_TEAM',
    right_on='HOME_TEAM_ABBREVIATION',
    how='left'
)
games_with_home

,GAME_ID,GAME_DATE,HOME_TEAM,AWAY_TEAM,HOME_WIN,HOME_TEAM_ABBREVIATION,HOME_W,HOME_L,HOME_W_PCT,HOME_FGM_RANK,...,HOME_BLK_RANK,HOME_PF_RANK,HOME_PTS_RANK,HOME_DivisionRank,HOME_HOME_WINS,HOME_ROAD_WINS,HOME_CurrentHomeStreak,HOME_CurrentRoadStreak,HOME_CurrentStreak,HOME_HighValueInjured
0,0022401195,2025-04-13,MIN,UTA,True,MIN,49,33,0.598,19,...,14,12,13,3,25,24,5,1,3,0
1,0022401189,2025-04-13,CLE,IND,False,CLE,64,18,0.780,4,...,27,11,1,1,34,30,-1,1,-1,0
2,0022401188,2025-04-13,BKN,NYK,False,BKN,26,56,0.317,30,...,26,28,29,4,12,14,-2,-1,-3,0
3,0022401197,2025-04-13,SAS,TOR,True,SAS,34,48,0.415,14,...,4,3,16,4,20,13,1,-1,1,1
4,0022401190,2025-04-13,MIA,WAS,False,MIA,37,45,0.451,24,...,30,1,24,3,19,17,-1,1,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,0022400071,2024-10-23,LAC,PHX,False,LAC,50,32,0.610,17,...,20,14,20,2,30,20,5,3,8,0
1226,0022400063,2024-10-23,DET,IND,False,DET,44,38,0.537,10,...,9,27,12,4,22,22,-1,-1,-2,0
1227,0022400072,2024-10-23,POR,GSW,False,POR,36,46,0.439,23,...,8,21,22,4,22,14,1,-2,1,0
1228,0022400062,2024-10-22,LAL,MIN,True,LAL,50,32,0.610,20,...,22,5,19,1,31,19,2,-1,-1,0


In [ ]:
final_games = games_with_home.merge(
    team_stats_df.add_prefix('AWAY_'),
    left_on='AWAY_TEAM',
    right_on='AWAY_TEAM_ABBREVIATION',
    how='left'
)
drop_columns = ['HOME_TEAM_ABBREVIATION','AWAY_TEAM_ABBREVIATION',]
final_games.drop(columns=drop_columns, inplace=True)
final_games.to_csv("final_games.csv",index=False)